In [3]:
# Compare the session folders that are found in 2 different parent folders.

import os

path_func = "/ptmp/hmueller2/Downloads/fmriprep_out/sub-06"
path_contrasts = "/ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-06"

# List all session folders in both directories
sessions_func = set([d for d in os.listdir(path_func) if d.startswith("ses-") and os.path.isdir(os.path.join(path_func, d))])
sessions_contrasts = set([d for d in os.listdir(path_contrasts) if d.startswith("ses-") and os.path.isdir(os.path.join(path_contrasts, d))])

# Find overlapping session folders
non_overlapping_func = sorted(sessions_func - sessions_contrasts)
non_overlapping_contrasts = sorted(sessions_contrasts - sessions_func)
print("Session folders only in func:", non_overlapping_func)
print("Session folders only in contrasts:", non_overlapping_contrasts)

Session folders only in func: ['ses-05', 'ses-06', 'ses-07', 'ses-13', 'ses-14', 'ses-22', 'ses-23', 'ses-32', 'ses-33', 'ses-34', 'ses-39']
Session folders only in contrasts: []


In [36]:
# Find files containing a kewoard in their names within a specified directory

import os

keywoard = "FingerTapping"
search_path = "/ptmp/hmueller2/Downloads/ibc_raw/sub-06"
raiders_files = []

for root, dirs, files in os.walk(search_path):
    for file in files:
        if keywoard in file:
            raiders_files.append(os.path.join(root, file))

print(f"Found {len(raiders_files)} files containing '{keywoard}':")
for f in raiders_files:
    print(f)

Found 6 files containing 'FingerTapping':
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-pa_run-01_sbref.nii.gz
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-pa_run-01_bold.nii.gz
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-ap_run-02_bold.nii.gz
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-ap_run-02_sbref.nii.gz
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-pa_run-01_events.tsv
/ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-ap_run-02_events.tsv


In [38]:
import pandas as pd

# Check the events files for these runs and ensure they contain the expected conditions (navigation, no_tap)

subject = "06"
session = "36"
task = "FingerTapping"
direction = "ap"
run = "02"
condition = "no_tap"

path_events = f'/ptmp/hmueller2/Downloads/ibc_raw/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-{task}_dir-{direction}_run-{run}_events.tsv'

# Load the events file
if os.path.exists(path_events):
    events_df = pd.read_csv(path_events, sep='\t')
    if condition in events_df['trial_type'].unique():
        print(f"Condition '{condition}' found in {path_events}")
    else:
        print(f"Condition '{condition}' NOT found in {path_events}")
else:
    print(f"Events file not found: {path_events}")

Condition 'no_tap' NOT found in /ptmp/hmueller2/Downloads/ibc_raw/sub-06/ses-36/func/sub-06_ses-36_task-FingerTapping_dir-ap_run-02_events.tsv


In [28]:
# Compare the number of timepoints in CIFTI files with the number of rows in motion files

import nibabel as nib

subject = "06"
session = "22"

path_func = f"/ptmp/hmueller2/Downloads/fmriprep_out/sub-{subject}"
regressors_dir = os.path.join(path_func, f"ses-{session}", "postfmriprep", "regressors")
glm_dir = os.path.join(path_func, f"ses-{session}", "postfmriprep", "GLM")

# List all CIFTI files and corresponding motion files for all runs
cifti_files = [f for f in os.listdir(glm_dir) if f.endswith(".nii") and f"sub-{subject}_ses-{session}_task-Audio" in f]
motion_files = [f for f in os.listdir(regressors_dir) if f.endswith("motion.txt") and f"sub-{subject}_ses-{session}_task-Audio" in f]

# Sort to align runs
cifti_files.sort()
motion_files.sort()

for cifti_file in cifti_files:
    # Extract run identifier from file name
    parts = cifti_file.split("_")
    run_part = [p for p in parts if p.startswith("run-")]
    if not run_part:
        continue
    run_id = run_part[0]
    # Find matching motion file
    motion_file = [f for f in motion_files if run_id in f]
    if not motion_file:
        print(f"No motion file found for {cifti_file}")
        continue
    motion_file = motion_file[0]
    # Load CIFTI file and get number of timepoints
    cifti_path = os.path.join(glm_dir, cifti_file)
    img = nib.load(cifti_path)
    n_timepoints = img.shape[0]
    # Load motion file and count rows
    motion_path = os.path.join(regressors_dir, motion_file)
    with open(motion_path, "r") as mf:
        n_rows = sum(1 for _ in mf)
    print(f"{cifti_file}: {n_timepoints} timepoints, {motion_file}: {n_rows} rows")


sub-06_ses-22_task-Audio_dir-ap_run-04_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-ap_run-04_motion.txt: 189 rows
sub-06_ses-22_task-Audio_dir-ap_run-05_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-ap_run-05_motion.txt: 189 rows
sub-06_ses-22_task-Audio_dir-ap_run-06_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-ap_run-06_motion.txt: 189 rows
sub-06_ses-22_task-Audio_dir-pa_run-01_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-pa_run-01_motion.txt: 189 rows
sub-06_ses-22_task-Audio_dir-pa_run-02_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-pa_run-02_motion.txt: 189 rows
sub-06_ses-22_task-Audio_dir-pa_run-03_cleaned_noscrub.dtseries.nii: 189 timepoints, sub-06_ses-22_task-Audio_dir-pa_run-03_motion.txt: 189 rows


In [3]:
# Count the number of contrast maps for each subject

import os
from collections import defaultdict

# Base path to the z-maps
base_path = "/ptmp/hmueller2/Downloads/contrast_maps_fsLR"
print("Base path:", base_path)

# Initialize a dictionary to store the count of contrast maps per subject
contrast_map_counts = defaultdict(int)

# Loop through all subjects and sessions
for subject in os.listdir(base_path):
    if subject.startswith("sub-15") and subject[4:].isdigit():  # Check subject naming pattern
        subject_path = os.path.join(base_path, subject)
        print("Subject path:", subject_path)
        if os.path.isdir(subject_path):  # Ensure it's a directory
            for session in os.listdir(subject_path):
                if session.startswith("ses-") and session[4:].isdigit():  # Check session naming pattern
                    session_path = os.path.join(subject_path, session)
                    # print("  Session path:", session_path)
                    if os.path.isdir(session_path):  # Ensure it's a directory
                        # Try to find all directories matching the pattern inside the session_path
                        for res_dir in os.listdir(session_path):
                            if res_dir.startswith("res_task-") and "fsLR" in res_dir and "dir-" in res_dir:
                                res_dir_path = os.path.join(session_path, res_dir)
                                print("    Res dir path:", res_dir_path)
                                z_score_maps_path = os.path.join(res_dir_path, "z_score_maps")
                                #print("      z_score_maps_path:", z_score_maps_path)
                                if os.path.exists(z_score_maps_path):
                                    # Count the number of .dscalar.nii files in the directory
                                    contrast_maps = [f for f in os.listdir(z_score_maps_path) if f.endswith(".dscalar.nii")]
                                    contrast_map_counts[subject] += len(contrast_maps)

# Print the results
for subject, count in contrast_map_counts.items():
    print(f"Subject {subject}: {count} contrast maps")

Base path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR
Subject path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-FaceBody_space-fsLR_run-04_dir-ap
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-FaceBody_space-fsLR_run-01_dir-pa
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-Scene_space-fsLR_run-02_dir-ap
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-Scene_space-fsLR_run-04_dir-ap
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-Scene_space-fsLR_run-03_dir-pa
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-Scene_space-fsLR_run-01_dir-pa
    Res dir path: /ptmp/hmueller2/Downloads/contrast_maps_fsLR/sub-15/ses-40/res_task-FaceBody_space-fsLR_run-02_dir-ap
    Res dir path: /ptmp/hmueller2/Downloads/contra

In [ ]:
# Count the number of contrast maps that overlap between all subjects

import os

base_path = "/ptmp/hmueller2/Downloads/contrast_maps_fsLR"
subject_contrast_maps = {}

# Loop through all subjects and collect contrast map filenames
for subject in os.listdir(base_path):
    if subject.startswith("sub-06") and subject[4:].isdigit():
        subject_path = os.path.join(base_path, subject)
        if os.path.isdir(subject_path):
            contrast_maps_set = set()
            for session in os.listdir(subject_path):
                if session.startswith("ses-") and session[4:].isdigit():
                    session_path = os.path.join(subject_path, session)
                    if os.path.isdir(session_path):
                        for res_dir in os.listdir(session_path):
                            if res_dir.startswith("res_task-") and "fsLR" in res_dir and "dir-" in res_dir:
                                z_score_maps_path = os.path.join(session_path, res_dir, "z_score_maps")
                                if os.path.exists(z_score_maps_path):
                                    maps = [f for f in os.listdir(z_score_maps_path) if f.endswith(".dscalar.nii")]
                                    contrast_maps_set.update(maps)
            subject_contrast_maps[subject] = contrast_maps_set

# Find overlap (intersection) of contrast map filenames across all subjects
if subject_contrast_maps:
    overlap_maps = set.intersection(*subject_contrast_maps.values())
    print(f"Number of overlapping contrast maps across all subjects: {len(overlap_maps)}")
    print("Overlapping contrast maps:")
    for fname in sorted(overlap_maps):
        print(fname)
else:
    print("No subjects found or no contrast maps available.")

Number of overlapping contrast maps across all subjects: 553
Overlapping contrast maps:
0back-2back.dscalar.nii
0back_body.dscalar.nii
0back_face.dscalar.nii
0back_place.dscalar.nii
0back_tools.dscalar.nii
2_dots-2_dots_control.dscalar.nii
2_dots.dscalar.nii
2_dots_control.dscalar.nii
2_letters_different-same.dscalar.nii
2_letters_different.dscalar.nii
2_letters_same.dscalar.nii
2back-0back.dscalar.nii
2back_body.dscalar.nii
2back_face.dscalar.nii
2back_place.dscalar.nii
2back_tools.dscalar.nii
4_dots-4_dots_control.dscalar.nii
4_dots.dscalar.nii
4_dots_control.dscalar.nii
4_letters_different-same.dscalar.nii
4_letters_different.dscalar.nii
4_letters_same.dscalar.nii
6_dots-2_dots.dscalar.nii
6_dots-6_dots_control.dscalar.nii
6_dots.dscalar.nii
6_dots_control.dscalar.nii
6_letters_different-2_letters_different.dscalar.nii
6_letters_different-same.dscalar.nii
6_letters_different.dscalar.nii
6_letters_same.dscalar.nii
accept-reject.dscalar.nii
achromatic.dscalar.nii
alphabet-silence.dsca

In [4]:
# Print the shape of the FPN_communities dscalar.nii file
import nibabel as nib  # Ensure nibabel is imported
import sys

FPN_communities = "/ptmp/hmueller2/Downloads/subnetworks/infomap/sub-06/FPN_communities.dscalar.nii"

img = nib.load(FPN_communities)
print("Shape of FPN_communities.dscalar.nii:", img.shape)

# Print the first 5 values of the FPN_communities dscalar.nii file
print("First 5 values in FPN_communities.dscalar.nii:", img.get_fdata()[:5])

Shape of FPN_communities.dscalar.nii: (1, 91282)
First 5 values in FPN_communities.dscalar.nii: [[0. 0. 0. ... 0. 0. 0.]]


In [9]:
import nibabel as nib
import numpy as np

subject = "05"
FPN_roi = "/ptmp/hmueller2/Downloads/individual_networks/sub-05/resting_state/Frontoparietal_roi.dscalar.nii"
data = nib.load(FPN_roi)
unique_labels = np.unique(data.get_fdata())

print("Shape of Frontoparietal_roi.dscalar.nii:", data.shape)
print("Unique labels in Frontoparietal_roi.dscalar.nii:", sorted(unique_labels))

Shape of Frontoparietal_roi.dscalar.nii: (1, 91282)
Unique labels in Frontoparietal_roi.dscalar.nii: [np.float64(0.0), np.float64(1.0)]


In [14]:
import nibabel as nib
import numpy as np

subject = "04"
cleaned_dtseries = "/ptmp/hmueller2/Downloads/individual_networks/sub-04/resting_state/sub-04_all-tasks_concatenated_cleaned_fsLR.dtseries.nii"
data = nib.load(cleaned_dtseries)
unique_labels = np.unique(data.get_fdata())

print("Shape of sub-04_all-tasks_concatenated_cleaned_fsLR.dtseries.nii:", data.shape)

Shape of sub-04_all-tasks_concatenated_cleaned_fsLR.dtseries.nii: (3656, 91282)


In [ ]:
import nibabel as nib
import numpy as np
import os

subject = "06"  # or whatever subject you're testing
working_dir = "/ptmp/hmueller2/Downloads"  # adjust to your path
half_dir = os.path.join(working_dir, 'individual_networks', f'sub-{subject}', 'resting_state')
subnetworks_dir = os.path.join(working_dir, 'subnetworks', 'infomap', f'sub-{subject}')

# Check the original files
filename = os.path.join(half_dir, f'sub-{subject}_all-tasks_concatenated_cleaned_fsLR.dtseries.nii')
networks_file = os.path.join(half_dir, 'Bipartite_PhysicalCommunities+AlgorithmicLabeling.dlabel.nii')
ptseries_path = os.path.join(half_dir, f'sub-{subject}_individual_nets_concat.ptseries.nii')
subnetworks_file = os.path.join(subnetworks_dir, 'FPN_communities.dscalar.nii')


print("=== File Shape Analysis ===")

# 1. Original dtseries
if os.path.exists(filename):
    dtseries_img = nib.load(filename)
    print(f"Original dtseries shape: {dtseries_img.shape}")
    print(f"Original dtseries header info: {dtseries_img.header}")

# 2. Atlas/networks file
if os.path.exists(networks_file):
    atlas_img = nib.load(networks_file)
    atlas_data = atlas_img.get_fdata()
    print(f"Atlas shape: {atlas_data.shape}")
    print(f"Unique values in atlas: {np.unique(atlas_data)}")
    print(f"Number of unique networks: {len(np.unique(atlas_data)) - 1}")  # -1 for background

# 3. wb_command generated ptseries
if os.path.exists(ptseries_path):
    ptseries_img = nib.load(ptseries_path)
    print(f"wb_command ptseries shape: {ptseries_img.shape}")
    print(f"wb_command ptseries header info: {ptseries_img.header}")
else:
    print("wb_command ptseries doesn't exist yet")

print("\n=== Manual Parcellation Analysis ===")

# 4. Manual parcellation result
if os.path.exists(filename) and os.path.exists(networks_file):
    dtseries_img = nib.load(filename)
    atlas_img = nib.load(networks_file)
    
    dtseries_data = dtseries_img.get_fdata()
    atlas_data = atlas_img.get_fdata().squeeze()[:64984]
    
    # Check what RR.cifti_parcellate returns
    sys.path.insert(1, '/home/hmueller2/ibc_code/ibc_latent/Preprocessing/Aradia')
    import RR_utils as RR
    
    dtseries_cortex = dtseries_data[:, :64984]
    all_data_concat = RR.cifti_parcellate(dtseries_cortex, atlas_data)
    
    print(f"Manual parcellation result shape: {all_data_concat.shape}")
    print(f"After removing FPN (8) and Noise (-1): {all_data_concat.shape[1] - 2}")

# 5. Subnetwork communities file
if os.path.exists(subnetworks_file):
    subnetworks_img = nib.load(subnetworks_file)
    subnetworks_data = subnetworks_img.get_fdata()
    print(f"Subnetwork communities shape: {subnetworks_data.shape}")
    print(f"Unique values in subnetwork communities: {np.unique(subnetworks_data)}")
    print(f"Number of unique communities: {len(np.unique(subnetworks_data)) - 1}")  # -1 for background

=== File Shape Analysis ===
Original dtseries shape: (2034, 91282)
Original dtseries header info: <nibabel.cifti2.cifti2.Cifti2Header object at 0x7f830813f6b0>
Atlas shape: (1, 91282)
Unique values in atlas: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 14. 15. 16. 17. 18.
 19. 20.]
Number of unique networks: 19
wb_command ptseries shape: (2034, 21)
wb_command ptseries header info: <nibabel.cifti2.cifti2.Cifti2Header object at 0x7f825bf43fe0>

=== Manual Parcellation Analysis ===


NameError: name 'sys' is not defined